In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
import mysql.connector

app = Flask(__name__)
app.secret_key = 'peter930512'  # 用于会话管理的密钥

# 配置你的 MySQL 连接
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='peter930512',
    database='專題用'
)

@app.route('/', methods=['GET', 'POST'])
def manage_users():
    cursor = conn.cursor(dictionary=True)

    # 添加用户
    if request.method == 'POST':
        if 'add' in request.form:  # 检查是否为添加用户请求
            name = request.form['name']
            email = request.form['email']
            cursor.execute("INSERT INTO Users (Name, Email) VALUES (%s, %s)", (name, email))
            conn.commit()
            flash('用户添加成功!')
        elif 'update' in request.form:  # 更新用户请求
            user_id = request.form['user_id']
            name = request.form['name']
            email = request.form['email']
            cursor.execute("UPDATE Users SET Name = %s, Email = %s WHERE UserID = %s", (name, email, user_id))
            conn.commit()
            flash('用户信息更新成功!')
        elif 'delete' in request.form:  # 删除用户请求
            user_id = request.form['user_id']
            cursor.execute("DELETE FROM Users WHERE UserID = %s", (user_id,))
            conn.commit()
            flash('用户删除成功!')

    # 查询所有用户
    cursor.execute("SELECT Users.UserID, Users.Name, Users.Email FROM Users;")
    user_results = cursor.fetchall()

    # 处理按姓名和产品名称筛选条件
    name_filter = request.args.get('name_filter', '')  # 从GET请求接收姓名筛选条件
    product_filter = request.args.get('product_filter', '')  # 从GET请求接收产品名称筛选条件
    query_params = []

    # INNER JOIN 查询（根据姓名和产品名称筛选）
    sql_query = """
        SELECT Users.UserID, Users.Name, Users.Email, Insurance_Products.ProductName, Recommendations.RecommendationReason, 
               Chatbot_Interactions.UserQuery, Chatbot_Interactions.BotResponse
        FROM Users
        INNER JOIN Recommendations ON Users.UserID = Recommendations.UserID
        INNER JOIN Insurance_Products ON Recommendations.ProductID = Insurance_Products.ProductID
        INNER JOIN Chatbot_Interactions ON Users.UserID = Chatbot_Interactions.UserID
    """
    filters = []

    if name_filter:  # 如果有提供姓名筛选
        filters.append("Users.Name LIKE %s")
        query_params.append(f"%{name_filter}%")  # 使用模糊匹配

    if product_filter:  # 如果有提供产品名称筛选
        filters.append("Insurance_Products.ProductName LIKE %s")
        query_params.append(f"%{product_filter}%")  # 使用模糊匹配

    if filters:  # 如果有任何过滤条件，则加上 WHERE 子句
        sql_query += " WHERE " + " AND ".join(filters)

    cursor.execute(sql_query, query_params)
    join_results = cursor.fetchall()
    cursor.close()

    return render_template('template1.html', user_results=user_results, join_results=join_results)

# 运行 Flask 应用
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.110.8:5000
Press CTRL+C to quit
192.168.110.8 - - [16/Oct/2024 10:44:53] "GET / HTTP/1.1" 200 -
